In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

/Users/joemccann/VENVs/fpENV/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [20]:
# 
def get_adv_stats(pos,week,year):
    url = 'https://www.fantasypros.com/nfl/advanced-stats-{}.php?week={}&range=week&year={}'.format(pos,week,year)
    response = requests.get(url)

    if response.status_code == 200:
        return response.text
    else:
        print("Failed to retrieve the page.")

def get_fts_pts(pos,year,week):
    url = 'https://www.fantasypros.com/nfl/stats/{}.php?year={}&week={}&range=week'.format(pos,year,week)
    response = requests.get(url)

    if response.status_code == 200:
        return response.text
    else:
        print("Failed to retrieve the page.")

In [37]:
adv_column_names = ['rank', 'player', 'games', 'att', 'yds', 'y/att', 'ybcon', 'ybon/att', 'rush_yacon', 'yacon/att', 'brktkl', 'tk_loss', 'tk_loss_yds', 'lng_td', '10_plus_yds', '20_plus_yds', '30_plus_yds', '40_plus_yds', '50_plus_yds', 'lng', 'rec', 'tgt', 'rz_tgt', 'rec_yacon']

def create_RB_weekly_report(pos, week, year, column_names):
    page_content = get_adv_stats(pos,week, year)
    soup = BeautifulSoup(page_content, 'html.parser')
    
    table = soup.find('table')
    
    table_data = []
    for row in table.find_all('tr'):
        row_data = [cell.text.strip() for cell in row.find_all('td')]
        table_data.append(row_data)
    
    report_df = pd.DataFrame(table_data)
    report_df.columns = adv_column_names
    report_df = report_df.drop([0, 1], axis=0)
    report_df.columns = [column.lower() for column in report_df.columns]
    report_df['week'] = week
    report_df['year'] = year
   
    return report_df

In [38]:
adv_stats = create_RB_weekly_report('rb',7,2023,column_names).head()

In [39]:
fts_columns = ['rank', 'player','att', 'yds_rush', 'yds_rush/a', 'lg', '20_plus', 'td_rush', 'rec', 'tgt', 'yds_rec', 'yds/rec', 'td_rec', 'rl', 'g', 'fpts', 'fpts/g', 'rost']
def rb_fantasy_pts_per_week(pos, year, week):
    page_content = get_fts_pts(pos, year, week)
    soup = BeautifulSoup(page_content, 'html.parser')
    
    table = soup.find('table')
    
    table_data = []
    for row in table.find_all('tr'):
        row_data = [cell.text.strip() for cell in row.find_all('td')]
        table_data.append(row_data)
    
    report_df = pd.DataFrame(table_data)
    report_df.columns = fts_columns
    report_df = report_df.drop([0, 1], axis=0)
    report_df.columns = [column.lower() for column in report_df.columns]
    report_df['week'] = week
    report_df['year'] = year
   
    return report_df

In [42]:
fts_pts = rb_fantasy_pts_per_week('rb',2023, 7).head()

In [43]:
fts_pts

,rank,player,att,yds_rush,yds_rush/a,lg,20_plus,td_rush,rec,tgt,yds_rec,yds/rec,td_rec,rl,g,fpts,fpts/g,rost,week,year
2,1,D'Onta Foreman (CHI),16,89,5.6,14,0,2,3,5,31,10.3,1,0,1,30.0,30.0,32.3%,7,2023
3,2,Gus Edwards (BAL),14,64,4.6,20,1,1,1,1,80,80.0,0,0,1,20.4,20.4,82.2%,7,2023
4,3,Travis Etienne Jr. (JAC),14,53,3.8,17,0,2,3,5,24,8.0,0,0,1,19.7,19.7,99.9%,7,2023
5,4,Christian McCaffrey (SF),15,45,3.0,9,0,1,3,3,51,17.0,1,1,1,19.6,19.6,100.0%,7,2023
6,5,Jahmyr Gibbs (DET),11,68,6.2,21,1,1,9,10,58,6.4,0,0,1,18.6,18.6,98.8%,7,2023


In [48]:
adv_stats = pd.merge(adv_stats, fts_pts[['fpts', 'fpts/g', 'rost']], left_on='rank', right_on='rank')

KeyError: 'rank'

In [ ]:
adv_stats.columns